In [1]:
!pip install torch==2.4.1 torchvision==0.19.1 --extra-index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 53.5 MB/s  0:00:09:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.8 MB/s  0:00:00
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 84.0 MB/s  0:00:00 eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 53.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.0 MB/s  0:00:00m0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 58.3 MB/s  0:00:07:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 MB 75.1 MB/s  0:00:04:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 75.7 MB/s  0:00:02:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 80.8 MB/s  0:00:00m0:00:0100:

In [2]:
!pip install -U git+https://github.com/luca-medeiros/lang-segment-anything.git

  Cloning https://github.com/luca-medeiros/lang-segment-anything.git to /tmp/pip-req-build-yw44lv9b
  Running command git clone --filter=blob:none --quiet https://github.com/luca-medeiros/lang-segment-anything.git /tmp/pip-req-build-yw44lv9b
  Resolved https://github.com/luca-medeiros/lang-segment-anything.git to commit 918043ed4666eea04da88aa179eb8d27ef4b1a1d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/facebookresearch/segment-anything-2 (to revision c2ec8e14a185632b0a5d8b161928ceb50197eddc) to /tmp/pip-install-m2omdr4e/sam-2_81ac27190b044421b71959de84e5665b
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything-2 /tmp/pip-install-m2omdr4e/sam-2_81ac27190b044421b71959de84e5665b
  Running command git rev-parse -q --verify 'sha^c2ec8e14a185632b0a5d8b161928ceb50197eddc'
  Running command git fetch -q https://github.co

In [8]:
# save as run_langsam_all_masks.py
from PIL import Image
import numpy as np
import torch
from datetime import datetime

from lang_sam import LangSAM  # https://github.com/luca-medeiros/lang-segment-anything


def _to_numpy_mask(m):
    """Convert mask to uint8 [0,255] HxW array."""
    if isinstance(m, torch.Tensor):
        m = m.detach().cpu().numpy()
    else:
        m = np.array(m)
    m = np.squeeze(m)

    if m.dtype == bool:
        m = m.astype(np.uint8) * 255
    elif m.dtype != np.uint8:
        m = (m >= 0.5).astype(np.uint8) * 255
    return m


def main():
    # 1) load inputs from current directory
    image_pil = Image.open("image.png").convert("RGB")
    prompt = [
        "mirror.chair.",
    ]

    # 2) init model
    model = LangSAM()

    # 3) predict
    results = model.predict([image_pil], prompt)

    if not results or "masks" not in results[0] or len(results[0]["masks"]) == 0:
        raise RuntimeError("No mask returned by LangSAM for the given prompt/image.")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 4) save all masks
    for i in range(len(results)):
        for idx, m in enumerate(results[i]["masks"]):
            mask_np = _to_numpy_mask(m)
            out_path = f"result/mask_{i}{idx}_{timestamp}.png"
            Image.fromarray(mask_np, mode="L").save(out_path)
            print(f"Saved {out_path}")


if __name__ == "__main__":
    main()


One or both local paths not provided. Loading from Hugging Face Hub: IDEA-Research/grounding-dino-base
Predicting 1 masks
Predicted 1 masks
Saved result/mask_00_20251120_032524.png
Saved result/mask_01_20251120_032524.png
Saved result/mask_02_20251120_032524.png


/tmp/ipykernel_2420204/2593918572.py:48: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(mask_np, mode="L").save(out_path)
